# Electro DC analysis

This example shows how to use PyAEDT to create a Maxwell DC analysis,
compute mass center, and move coordinate systems.

Keywords: **Maxwell 3D**, **DC**.

## Perform imports and define constants

Perform required imports.

In [1]:
import os
import tempfile
import time

from ansys.aedt.core import Maxwell3d
from ansys.aedt.core.generic.constants import SolutionsMaxwell3D


Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Launch AEDT

Create an instance of the ``Maxwell3d`` class named ``m3d`` by providing
the project name, the version, and the graphical mode.

In [4]:
project_name = os.path.join(temp_folder.name, "conductor_example.aedt")
m3d = Maxwell3d(
    project=project_name,
    version=AEDT_VERSION,
    new_desktop=True,
    non_graphical=NG_MODE,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_04698b1d-1e90-466c-b5b3-a94df215b834.log is enabled.


...

PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project conductor_example has been created.


PyAEDT INFO: No design is present. Inserting a new design.


PyAEDT INFO: Added design 'Maxwell 3D_0HN' of type Maxwell 3D.


PyAEDT INFO: Aedt Objects correctly read


## Set up Maxwell solution

Set up the Maxwell solution to DC.

In [5]:
m3d.solution_type = SolutionsMaxwell3D.ElectroDCConduction

## Create conductor

Create a conductor using copper, a predefined material in the Maxwell material library.

In [6]:
conductor = m3d.modeler.create_box(origin=[7, 4, 22], sizes=[10, 5, 30], name="Conductor", material="copper")

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


## Create setup and assign voltage

In [7]:
m3d.assign_voltage(assignment=conductor.faces, amplitude=0)
m3d.create_setup()

PyAEDT INFO: Boundary Voltage Voltage_Q6S6PK has been created.


PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpde17j77r.ansys\conductor_example.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpde17j77r.ansys\conductor_example.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.01598072052001953


MySetupAuto with 0 Sweeps

## Solve setup

In [8]:
m3d.analyze(cores=NUM_CORES)

PyAEDT INFO: Project conductor_example Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 3D correctly changed.


PyAEDT INFO: Solving all design setups. Analysis started...


PyAEDT INFO: Design setup None solved correctly in 0.0h 0.0m 20.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 3D correctly changed.


True

## Compute mass center

Compute mass center using PyAEDT advanced fields calculator.

In [9]:
scalar_function = ""
mass_center = {
    "name": "CM_X",
    "description": "Mass center computation",
    "design_type": ["Maxwell 3D"],
    "fields_type": ["Fields"],
    "primary_sweep": "distance",
    "assignment": "",
    "assignment_type": ["Solid"],
    "operations": [
        scalar_function,
        "EnterVolume('assignment')",
        "Operation('VolumeValue')",
        "Operation('Mean')",
    ],
    "report": ["Data Table"],
}
scalar_function = "Scalar_Function(FuncValue='X')"
mass_center["operations"][0] = scalar_function
m3d.post.fields_calculator.add_expression(mass_center, conductor.name)
mass_center["name"] = "CM_Y"
scalar_function = "Scalar_Function(FuncValue='Y')"
mass_center["operations"][0] = scalar_function
m3d.post.fields_calculator.add_expression(mass_center, conductor.name)
mass_center["name"] = "CM_Z"
scalar_function = "Scalar_Function(FuncValue='Z')"
mass_center["operations"][0] = scalar_function
m3d.post.fields_calculator.add_expression(mass_center, conductor.name)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpde17j77r.ansys\conductor_example.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpde17j77r.ansys\conductor_example.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.024695634841918945


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


'CM_Z'

## Get mass center

Get mass center using the fields calculator.

In [10]:
xval = m3d.post.get_scalar_field_value(quantity="CM_X")
yval = m3d.post.get_scalar_field_value(quantity="CM_Y")
zval = m3d.post.get_scalar_field_value(quantity="CM_Z")

PyAEDT INFO: Exporting CM_X field. Be patient


PyAEDT INFO: Quantity CM_X not present. Trying to get it from Stack


PyAEDT INFO: Exporting CM_Y field. Be patient


PyAEDT INFO: Quantity CM_Y not present. Trying to get it from Stack


PyAEDT INFO: Exporting CM_Z field. Be patient


PyAEDT INFO: Quantity CM_Z not present. Trying to get it from Stack


## Create variables

Create variables with mass center values.

In [11]:
m3d[conductor.name + "x"] = str(xval * 1e3) + "mm"
m3d[conductor.name + "y"] = str(yval * 1e3) + "mm"
m3d[conductor.name + "z"] = str(zval * 1e3) + "mm"

## Create coordinate system

Create a parametric coordinate system.

In [12]:
cs1 = m3d.modeler.create_coordinate_system(
    origin=[conductor.name + "x", conductor.name + "y", conductor.name + "z"],
    reference_cs="Global",
    name=conductor.name + "CS",
)

## Release AEDT

In [13]:
m3d.save_project()
m3d.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project conductor_example Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [14]:
temp_folder.cleanup()